# 04. Semantic Kernel

In this lab, we will introduce **Semantic Kernel**. Like **Langchain**, it provides a framework for working with AI models and supports applications written in Python. Semantic Kernel also adds support for .NET and Java applications.

As with the other labs, we'll start by reading in values from the `.env` file.

___
**NOTE**: This lab uses **.NET** so be sure to select .NET and not Python when starting to use this notebook.
___

In [1]:
#r "nuget: dotenv.net, 3.1.2"

using dotenv.net;

// Read values from .env file
var envVars = DotEnv.Fluent()
    .WithoutExceptions()
    .WithEnvFiles("../../../.env")
    .WithTrimValues()
    .WithDefaultEncoding()
    .WithOverwriteExistingVars()
    .WithoutProbeForEnv()
    .Read();

// Load values into variables and strip quotes
var model = envVars["AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME"].Replace("\"", "");
var azureEndpoint = envVars["AZURE_OPENAI_ENDPOINT"].Replace("\"", "");
var apiKey = envVars["AZURE_OPENAI_API_KEY"].Replace("\"", "");


Installed Packages dotenv.net, 3.1.2

To start working with Semantic Kernel, we'll need to load its nuget package.

In [2]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"

Installed Packages Microsoft.SemanticKernel, 1.0.1

Semantic Kernel works by creating an instance of the Kernel and then adding in various plugins to perform different functions. Those addins or functions can then be called individually or chained together to perform more complex tasks.

We use the standard .NET `builder` pattern to initialise the kernel. Notice that we pass in the details of the completion model that we're going to use, the Azure OpenAI API endpoint URL and the API key.

In [3]:
using Microsoft.SemanticKernel;

var builder = Kernel.CreateBuilder();
builder.Services.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
var kernel = builder.Build();

Let's create a Semantic Function to perform a simple request to Azure OpenAI. In this case, the function contains a *prompt template*. The template allows us to define a prompt and add placeholders for values that we will provide later. These values could come from user input, or another function, for example.

In [4]:
var whatCanIMakeFunction = kernel.CreateFunctionFromPrompt(
    new PromptTemplateConfig()
    {
        Template = @"What interesting things can I make with a {{$item}}?",
        InputVariables = [
            new() { Name = "item", Description = "An item to make something with.", IsRequired=true }
        ]
    });


`{{$item}}` represents the value we will provide later and will be replaced when we make the call to Azure OpenAI.

Next, we'll define a value for `item` and then call the function, passing in the `item` as we do so.

In [5]:
string item = "raspberry pi";

var response = await kernel.InvokeAsync(whatCanIMakeFunction, new () { { "item", item }});

Console.WriteLine(response);

A Raspberry Pi is a versatile, affordable, credit-card-sized computer that can be used for an array of creative and educational projects. Here’s a list of interesting things you can do with a Raspberry Pi:

1. Home Automation System: Use your Pi to control lights, thermostats, and other smart devices in your home.

2. Media Center: Transform your Pi into a media center with software like OSMC or Plex, allowing you to stream content to your TV.

3. Retro Gaming Console: Emulate old gaming consoles and play classic games by setting up RetroPie or Recalbox.

4. Personal Web Server: Host your own website or blog by turning the Raspberry Pi into a LAMP (Linux, Apache, MySQL, PHP) or LEMP (Linux, Nginx, MySQL, PHP) server.

5. Ad Blocking Server: Set up Pi-hole on your Raspberry Pi to block ads across your entire home network.

6. Network Attached Storage (NAS): Create your own storage server for backups and file sharing.

7. Home Security System: Use the Pi with cameras and motion sensors t

The result is a list of ideas for things we could make with a Raspberry Pi. 

We could then use Semantic Kernel to take that list of ideas and summarize it. First, we'll create a new prompt that will generate a summary of some text.

In [6]:
var thingsToMakeSummary = kernel.CreateFunctionFromPrompt(
    new PromptTemplateConfig()
    {
        Template = @"Summarize the following text: {{$thingsToMake}}?",
        InputVariables = [
            new() { Name = "thingsToMake", Description = "A list of things you could make.", IsRequired=true }
        ]
    });

Next, we can call the kernel again, this time passing in the list of ideas, the `response` from the previous call, as the value for `thingsToMake`.

In [7]:
var summary = await kernel.InvokeAsync(thingsToMakeSummary, new () { { "thingsToMake", response }});

Console.WriteLine(summary);

The Raspberry Pi is a small, affordable computer that can be used for a wide range of projects. Some examples include home automation, creating a media center, retro gaming, web hosting, ad blocking, setting up a NAS, home security, building a weather station, learning to code, robotics, digital art, running a VPN, experimenting with AI, IoT projects, amateur radio, cluster computing, streaming music, making a smart mirror, creating time-lapse videos, developing a voice assistant, and monitoring plants. The possibilities depend on the model of Raspberry Pi and may require extra hardware. Online resources are available for guidance.


## Summary

Semantic Kernel is another example of an AI orchestrator. Like other orchestrators, it can be used to simplify the process of creating complex AI applications.

## Next Section

📣 [AI Orchestration](../../03-orchestration/README.md)